# LSE Machine Learning: Practical Applications
## Module 7 Unit 2 IDE Activity (Practice) | Practise fitting a neural network
### In this IDE activity, a neural network is trained to calculate predictions from the Boston housing data set.
As you complete this activity, you are required to read the text cells throughout the notebook and then run the code in the cells that follow. Be mindful of the syntax used to execute certain functionalities within R to produce a desired result. In completing this activity, you should gain the necessary practical skills to complete the IDE Activity (Assessment) that follows.

### Step 1: Load and install the relevant R packages

The first step is to load the necessary packages. For this example, tidyverse and caret are used. Tidyverse is used for data manipulation, and caret is used for hyperparameter tuning and cross-validation.

In [1]:
options(warn=-1)

In [2]:
library(tidyverse)
library(caret)
set.seed(123)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.4
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




### Step 2: Load the data

Load the Boston Housing data set using the `read_csv()` function.



In [3]:
data <- read_csv("BostonHousing.csv")


── Column specification ────────────────────────────────────────────────────────
cols(
  crim = col_double(),
  zn = col_double(),
  indus = col_double(),
  chas = col_double(),
  nox = col_double(),
  rm = col_double(),
  age = col_double(),
  dis = col_double(),
  rad = col_double(),
  tax = col_double(),
  ptratio = col_double(),
  lstat = col_double(),
  medv = col_double()
)




Once the data has been loaded into R, it must be split into training and test sets. To do this, use the `createDataPartition()` function, splitting the data into 70% training data and 30% test data.

In [4]:
trainIndex <- createDataPartition(data$medv, p=.7, list=F)
dataTrain <- data[trainIndex, ]
dataTest <- data[-trainIndex, ]

Fit a regression model on the training data set.

In [5]:
trainReg <- lm(medv ~ ., data = dataTrain)

Once the regression model is fitted onto the data set, use it to calculate a prediction on the test data set.

In [6]:
testRegPred <- predict(trainReg, newdata = dataTest)

From there, the RMSE can now be calculated.

In [7]:
regRMSE <- sqrt(mean((testRegPred - dataTest$medv)^2))
regRMSE

[1] 4.585006

This RMSE can later be compared to the RMSE value of the neural network.

### Step 3: Set up the model parameters

Once the data is loaded and split, the control for the cross-validation process is set up. For this example, cross-validation with ten folds is used.

In [8]:
tuneCtrl <- trainControl(method = "cv", n = 10)

Next, set up the grid parameters of the neural network. For the grid of hyperparameters, neural networks with a single hidden layer, in addition to the layers of the inputs and the output of 6, 8, 10, 12, or 24 nodes, are tested. The dropout rate is also varied.

In [9]:
nnetGrid <- expand.grid(size = c(6, 8, 10, 12, 24),
                        decay = c(0.01, 0.001, 0.1))

The components can now be used to train the model.

### Step 4: Train the model

Train the model to estimate the median value, using RMSE as the performance metric. Remember, the linout parameter needs to be equal to 1, or true, as this is a regression problem and not a classification problem. Medv is the predicted variable. The method that is used is the `nnet` function, and RMSE as the metric to minimise. TuneGrid and trControl are the objects created in the previous step.

In [10]:
nnetFit <- train(medv ~ ., 
                data = dataTrain,
                method = "nnet",
                metric = "RMSE",
                tuneGrid = nnetGrid,
                trControl = tuneCtrl,
                maxit = 200,
                linout = TRUE, 
                trace = FALSE)

The model has now been trained and can be fitted onto the test data set.

### Step 5: Construct a response to the problem

With the model fitted, the output can be predicted on the test data and RMSE calculated. This is done using the `predict()` function and then calculating the square root of the average difference between the predicted value and the actual values squared.

In [11]:
nnetPredict <- predict(nnetFit, newdata = dataTest)
nnetRMSE <- sqrt(mean((nnetPredict - dataTest$medv)^2))

Compare regression RMSE fitted earlier and the neural network RMSE.

In [12]:
regRMSE
nnetRMSE

[1] 4.585006

[1] 3.43256

The neural network performs better than the regression model, as indicated by the smaller RMSE value. This is because a neural network model, unlike the linear regression model, provides a flexible model that allows for highly non-linear and complex associations between the inputs and output. This is typically the case in reality: Very few things are actually linear and additive. Neural networks are therefore more suitable for approximating how things really are.